## CET3003 Week 6 Workshop
### Template for tasks

### Task 3

#### Install and import required libraries

In [2]:
# just execute this cell if the libraries have NOT been installed
# uncomment if needed

!py -m pip install scikit-image       
#!pip install numpy scikit-image          # MAC

'py' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
import numpy as np
import cv2
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt
from os import makedirs

ModuleNotFoundError: No module named 'numpy'

In [4]:
# global variables
qualities = [90, 70, 40, 20]
size = 256
img_folder = 'img'

# create a new directoy for storing the created images
makedirs(img_folder, exist_ok=True)

In [6]:
# generate a image using simple PCG
def generate_procedural_image(size=256):
    # create empty image
    img = np.zeros((size, size, 3), dtype=np.uint8)

    # horizontal gradient for R channel
    for x in range(size):
        img[:, x, 2] = int((x / (size-1)) * 255)  # R channel

    # vertical gradient for G channel
    for y in range(size):
        img[y, :, 1] = int((y / (size-1)) * 255)  # G channel

    # add low-frequency noise for B channel
    freq = 8  # lower frequency = smoother
    y, x = np.meshgrid(np.linspace(0, freq*np.pi, size), np.linspace(0, freq*np.pi, size))
    img[:, :, 0] = ((np.sin(x) + np.sin(y)) / 2 * 127 + 128).astype(np.uint8)  # B channel

    return img

# generate a hand-made image
def generate_handmade_image(size=256):
    # create a black RGB image
    img = np.zeros((size, size, 3), dtype=np.uint8)

    # large central square (light blue)
    cv2.rectangle(img, (size//4, size//4), (3*size//4, 3*size//4), (180, 220, 255), -1)

    # circle in top-left quadrant (pink)
    cv2.circle(img, (size//4, size//4), size//8, (255, 180, 200), -1)

    # diagonal line (yellow)
    cv2.line(img, (0, 0), (size-1, size-1), (255, 255, 0), 3)

    # smaller squares in corners (green)
    offset = size // 16
    cv2.rectangle(img, (offset, offset), (2*offset, 2*offset), (0, 255, 0), -1)
    cv2.rectangle(img, (size-2*offset, offset), (size-offset, 2*offset), (0, 255, 0), -1)
    cv2.rectangle(img, (offset, size-2*offset), (2*offset, size-offset), (0, 255, 0), -1)
    cv2.rectangle(img, (size-2*offset, size-2*offset), (size-offset, size-offset), (0, 255, 0), -1)

    # add a horizontal gradient strip
    for i in range(size):
        cv2.line(img, (0, i), (size-1, i), (i % 256, 128, 255-i % 256), 1)

    return img

# save images as PNG and JPEG using OpenCV
def save_images(img, base_name):
    cv2.imwrite(f"{img_folder}/{base_name}.png", img)  # PNG (lossless)
    for quality in qualities:
        cv2.imwrite(f"{img_folder}/{base_name}_q{quality}.jpg", img, [int(cv2.IMWRITE_JPEG_QUALITY), quality])

In [ ]:
# compute MSE and SSIM per channel
def compare_images(original, compressed):
    """
    Compute MSE and SSIM per channel between two images.
    Both inputs are expected in RGB uint8 format and same shape.
    Returns (mse_vals, ssim_vals) where each is a list of 3 floats for R,G,B.
    """
    # ensure float for MSE calc, but SSIM expects uint8 (we cast per-channel)
    mse_vals = []
    ssim_vals = []

    # for the R channel (the first channel)
    for c in range(3):
        diff = original[:, :, 0] - compressed[:, :, 0]
        sq_diff = diff * diff
        mse = np.mean(sq_diff)
        ssim_channel = ssim(original[:, :, 0], compressed[:, :, 0])

        mse_vals.append(mse)
        ssim_vals.append(ssim_channel)


    return mse_vals, ssim_vals

In [ ]:
'''
TODO:
    1. generate and save images using the above functions
    2. reload JPEGs for comparison
    3. compute MSE and SSIM between the procedural image
       and the compressed ones and display the metrics
    4. compute MSE and SSIM between the handmade image
       and the compressed ones and display the metrics
'''
proc_image = generate_procedural_image(size=256)
hm_image = generate_handmade_image(size=256)

# save images at different quality levels
save_images(proc_image, "procedural")
save_images(hm_image, "handmade")

proc_jpeg_imgs = []

for q in qualities:
    # load/import an image
    img = cv2.imread(f"{img_folder}/procedural_q{q}.jpg")
    proc_jpeg_imgs.append(img)

handmade_jpeg_imgs = []

for q in qualities:
    # load/import an image
    img = cv2.imread(f"{img_folder}/handmade_q{q}.jpg")
    handmade_jpeg_imgs.append(img)

In [ ]:
# proc image

for jpegimg in proc_jpeg_imgs: 
    mse, ssim = compare_images(proc_image, jpegimg)

    print(mse, ssim)

    for jpegimg in handmade_jpeg_imgs: 
        mse, ssim = compare_images(proc_image, jpegimg)

    print(mse, ssim)

NameError: name 'proc_jpeg_imgs' is not defined

In [ ]:
'''
    TODO: display the original images and the compressed versions
          using OpenCV
          refer to the in-class examples for more information
'''

### Task 5

In [ ]:
# import necessary libraries. you may need to install the missing libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import cv2

In [ ]:
# global variables. feel free to change any of them
img_folder = 'img/all_tasks'
output_folder = 'img/task5'
makedirs(output_folder, exist_ok=True)      # create the output folder

img = f'{img_folder}/lenna.png'
img_name = img.split('/')[-1].split('.')[0] # extract the file name (w/out extension)

k = 2
output_scale = 0.25  # 25% of original size

In [ ]:
def clustering_posterized_img(img, k = 8, output_scale=1):
    '''
    TODO:
        1. load an example image using OpenCV (cv2) and convert BGR to RGB
        2. reshape pixels for clustering
        3. cluster pixel colors using scikit-learn's KMeans function
           refer to the workshop manual for more information
        4. get the assigned colors from kmeans.cluster_centers_
        5. quantize the assigned colors to ensure that they are in the range of 0 - 255
           you may use .astype along with np.uint8 to convert the values        
        6. resize the output image
        7. return the output image
    '''

In [ ]:
output = clustering_posterized_img(img, k=k, output_scale=output_scale)

# TODO: save image using OpenCV
# cv2.imwrite expects BGR, so we need to convert back

### Task 6

In [ ]:
# import necessary libraries. you may need to install the missing libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import cv2

In [ ]:
# global variables. feel free to change any of them
img_folder = 'img/all_tasks'
output_folder = 'img/task6'
makedirs(output_folder, exist_ok=True)      # create the output folder

img = f'{img_folder}/lenna.png'
img_name = img.split('/')[-1].split('.')[0] # extract the file name (w/out extension)

# feel free to play around with these variables
patch_size = 8
k = 8

In [ ]:
def clustering_pixel_art_img(filename, patch_size=8, k=8):
    '''
    TODO:
        1. copy the script you implemented for task 5
        2. before the return statement, implement the following
            a. obtain the dimension of the posterized image using .shape
            b. make a copy of the posterized image using .copy
               you may call it img_mosaic
            c. for each patch (you may refer to the argument patch_size)
               compute the mean of the RGB value as the new color
               to proceed to the next patch, you may increment by patch_size
               you may also find np.mean useful for manipulating numpy arrays
            d. assign the new color to the corresponding position in img_mosaic
        3. return img_mosaic
    '''

In [ ]:
output = clustering_pixel_art_img(img, patch_size=patch_size, k=k)

# TODO: save image using OpenCV
# cv2.imwrite expects BGR, so we need to convert back


In [ ]:
# helper workflow to generate, save, reload, compute metrics and display
if __name__ == "__main__":
    import cv2
    import matplotlib.pyplot as plt
    from os import path

    # generate images (these functions exist earlier in the notebook)
    proc_img = generate_procedural_image(size=size)   # produced in BGR-like layout used by the template
    hand_img = generate_handmade_image(size=size)

    # Save originals and JPEGs
    save_images(proc_img, "procedural")
    save_images(hand_img, "handmade")

    def load_rgb(p):
        img = cv2.imread(p)               # cv2 reads BGR
        if img is None:
            raise FileNotFoundError(p)
        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # original images converted to RGB for comparison / display
    orig_proc_rgb = cv2.cvtColor(proc_img, cv2.COLOR_BGR2RGB)
    orig_hand_rgb = cv2.cvtColor(hand_img, cv2.COLOR_BGR2RGB)

    # compare and display results
    for base, orig in [("procedural", orig_proc_rgb), ("handmade", orig_hand_rgb)]:
        print(f"\n--- {base.upper()} ---")
        # show original once
        plt.figure(figsize=(4,4))
        plt.imshow(orig)
        plt.axis('off')
        plt.title(f"{base} original")
        plt.show()

        for q in qualities:
            jp = f"{img_folder}/{base}_q{q}.jpg"
            comp = load_rgb(jp)

            mse_vals, ssim_vals = compare_images(orig, comp)
            print(f"Quality {q}: MSE (R,G,B) = {[round(v,2) for v in mse_vals]} | SSIM (R,G,B) = {[round(v,4) for v in ssim_vals]}")

            # display original and compressed side-by-side
            fig, axes = plt.subplots(1,2, figsize=(8,4))
            axes[0].imshow(orig); axes[0].set_title("Original"); axes[0].axis('off')
            axes[1].imshow(comp); axes[1].set_title(f"JPEG q={q}\nMSE={sum(mse_vals)/3:.1f}, SSIM={sum(ssim_vals)/3:.3f}"); axes[1].axis('off')
            plt.show()